# Multi-Agents Orchestration Notebook

Ce notebook illustre un pipeline complet pour orchestrer la collaboration de plusieurs agents (Coder, Reviewer, Admin) autour d’un **notebook cible** à finaliser.

---

### Étapes principales

1. **Sélection du notebook à finaliser** (via un prompt ou dropdown `ipywidgets`).
2. **Objectif facultatif** : l’utilisateur peut préciser un objectif global qui sera injecté en contexte si présent.
3. **Chargement d’un état partagé** (`NotebookState`) permettant :
   - de conserver un cache d’exécution (pour éviter de relancer Papermill après chaque modification si le notebook n’a pas changé)  
   - de gérer les modifications successives du `Coder`  
4. **Définition des plugins** (Coder, Reviewer, Admin) :
   - Coder : `update_cell`  
   - Reviewer : `get_notebook_status`  
   - Admin : `submit_for_approval`
5. **Création d’un `AgentGroupChat`** multi-agents sous Semantic Kernel, plus un système de **function calling** centralisé pour manipuler le notebook.
6. **Boucle itérative** gérée par des stratégies de sélection et de terminaison (Semantic Kernel) pour arrêter la conversation une fois le notebook approuvé.

---


In [58]:
# Cellule Code : Installation
%pip install papermill nbformat ipywidgets semantic-kernel --quiet


Note: you may need to restart the kernel to use updated packages.


### Configuration du Logger

Ici, nous activons un logger plus verbeux, avec couleurs, pour afficher toutes les actions (modifications du notebook, appels de fonctions, messages entre agents, etc.).


In [59]:
# Cellule Code : Imports et configuration
import os
import json
import logging
import nbformat
import papermill as pm
from datetime import datetime

# Pour la conversation multi-agents
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# On importe la base pour nos stratégies
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole

# Configuration d'un logger simple
logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)
if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s")
    ch.setFormatter(formatter)
    logger.addHandler(ch)

logger.info("Logger ready. Imports done.")


2025-02-10 07:58:07,718 [INFO] Orchestration - Logger ready. Imports done.


## Sélection du notebook à finaliser et Objectif facultatif

In [60]:
# Cellule Code : Sélection notebook

# On suppose que vous êtes dans un répertoire contenant "Notebook-Template.ipynb"
DEFAULT_TEMPLATE = "Notebook-Template.ipynb"



target_notebook = DEFAULT_TEMPLATE


logger.info(f"Notebook cible: {target_notebook}")


2025-02-10 07:58:07,731 [INFO] Orchestration - Notebook cible: Notebook-Template.ipynb


### Définition de l'état du Notebook: la classe NotebookState


In [61]:
class NotebookState:
    """
    Gère le JSON d'un notebook Jupyter, permet:
    - lecture du contenu complet,
    - mise à jour d'une cellule spécifique,
    - exécution du notebook via papermill (en lazy)
    - statut d'approbation final
    """

    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._cached_notebook = None
        self._dirty = False  # indique si des modifications ont été faites
        self._last_exec_path = None

        # Nouveau: champ d'approbation
        self._is_approved = False

        self._load_notebook()

    def _load_notebook(self):
        """Charge le notebook et met en cache son contenu JSON."""
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")
        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)
        logger.debug(f"Notebook '{self.notebook_path}' chargé, nb de cellules={len(self._cached_notebook.cells)}.")

    def get_notebook_json(self) -> str:
        """
        Retourne le JSON complet du notebook.
        Si le notebook est "sale" (dirty), on exécute avant pour actualiser outputs.
        """
        if self._dirty:
            self.execute_notebook()
        return json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)

    def update_cell(self, cell_index: int, new_source: str):
        """Met à jour la source d'une cellule par index."""
        if cell_index < 0 or cell_index >= len(self._cached_notebook.cells):
            raise IndexError(f"Cell index {cell_index} invalide.")
        old = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source
        self._dirty = True
        logger.info(f"[NotebookState] Cellule {cell_index} mise à jour.\n---OLD---\n{old}\n---NEW---\n{new_source}\n")

    def execute_notebook(self):
        """Exécute le notebook via papermill, puis recharge en mémoire."""
        logger.info(f"[NotebookState] Execution Papermill sur {self.notebook_path}...")
        output_path = self.notebook_path.replace(".ipynb", "-EXEC.ipynb")
        pm.execute_notebook(self.notebook_path, output_path, kernel_name="python3", progress_bar=False)
        # On remplace le notebook original par la sortie
        os.replace(output_path, self.notebook_path)
        self._load_notebook()
        self._dirty = False
        self._last_exec_path = self.notebook_path
        logger.info("[NotebookState] Execution OK, notebook mis à jour.")

    # --- Nouvelle partie : statut d'approbation ---
    def is_approved(self) -> bool:
        return self._is_approved

    def set_approved(self, approved: bool = True):
        self._is_approved = approved
        logger.info(f"[NotebookState] set_approved({approved}).")


### Test du Notebook state

In [62]:
# on crée une instance de NotebookState pour le notebook cible
notebook_state = NotebookState(target_notebook)
# on affiche le contenu du notebook
logger.info("Notebook chargé, voici son contenu:")
logger.info(notebook_state.get_notebook_json())

# On modifie une cellule à partir de son contenu

# on cherche la cellule contenant le placeholder {{TASK_DESCRIPTION}}




2025-02-10 07:58:07,759 [DEBUG] Orchestration - Notebook 'Notebook-Template.ipynb' chargé, nb de cellules=11.
2025-02-10 07:58:07,761 [INFO] Orchestration - Notebook chargé, voici son contenu:
2025-02-10 07:58:07,762 [INFO] Orchestration - {
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": "# Notebook de travail\n\nCe notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous."
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": "## 0. Objectif du Notebook\n\nDans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.\n\n### Tâche originale\n\nVoilà la tâche telle qu'elle a été initialement formulée :\n\n{{TASK_DESCRIPTION}}\n\nTASK_INSTRUCTION_PLACEHOLDER\n\n### Interprétation et sous-objectifs\n\nDécrire ici l'interprétation de la tâche et les étapes prévues pour la réaliser."
    },
    {
      "cell_type": "markdown",
      "metad

### Plugins


In [63]:
# Cellule Code : Plugins

from semantic_kernel.functions import kernel_function

class BaseNotebookPlugin:
    """Classe de base pour manipuler NotebookState."""

    def __init__(self, state: NotebookState):
        self.state = state

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook complet au format JSON."
    )
    def get_notebook_content(self) -> str:
        return self.state.get_notebook_json()


class CoderNotebookPlugin(BaseNotebookPlugin):
    """Permet de modifier la cellule d'un notebook."""

    @kernel_function(
        name="update_cell",
        description="Met à jour la source d'une cellule (par son index)."
    )
    def update_cell(self, cell_index_str: str, new_source: str) -> str:
        cell_index = int(cell_index_str)
        self.state.update_cell(cell_index, new_source)
        return f"Cellule {cell_index} mise à jour."


class ReviewerNotebookPlugin(BaseNotebookPlugin):
    """Permet de valider (ré-exécuter) le notebook."""

    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook via papermill, renvoie un résumé."
    )
    def validate_notebook(self) -> str:
        self.state.execute_notebook()
        return "Notebook exécuté et (potentiellement) validé."


class AdminNotebookPlugin(BaseNotebookPlugin):
    """Permet de donner l'approbation finale du notebook."""

    def __init__(self, state: NotebookState):
        super().__init__(state)

    @kernel_function(
        name="approve_notebook",
        description="Marque le notebook comme approuvé."
    )
    def approve_notebook(self) -> str:
        self.state.set_approved(True)
        return "Notebook APPROVED."


# Cellule Markdown : Stratégies


In [64]:
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=10)  # Limite de sécurité

    def __init__(self, state: NotebookState, max_steps: int = 10):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history):
        self._current_step += 1
        if self._current_step >= self._max_steps:
            logger.warning(f"Maximum steps ({self._max_steps}) reached. Terminating conversation.")
            return True
        is_approved = self._state.is_approved()
        logger.debug(f"Termination check - Approved: {is_approved}, Step: {self._current_step}/{self._max_steps}")
        return is_approved



class NotebookAwareSelectionStrategy(SelectionStrategy):
    _state: NotebookState = PrivateAttr()
    _index: int = PrivateAttr(default=-1)

    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state
        self._index = -1

    def reset(self) -> None:
        self._index = -1

    async def select_agent(self, agents, history):
        if self._state.is_approved():
            logger.info("Notebook approuvé => plus de sélection.")
            return None
        if not agents:
            return None
        self._index = (self._index + 1) % len(agents)
        return agents[self._index]


## Cellule Markdown : Création des agents


In [65]:
# Cellule Code : Agents

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    k = Kernel()
    
    k.add_service(OpenAIChatCompletion(service_id="default"))
    # Ajout du plugin
    k.add_plugin(plugin_instance, plugin_name="notebook_plugin")
    return k

# Préparation de l'état partagé
shared_state = NotebookState(notebook_path=target_notebook)

# Instanciation des plugins
coder_plugin = CoderNotebookPlugin(shared_state)
reviewer_plugin = ReviewerNotebookPlugin(shared_state)
admin_plugin = AdminNotebookPlugin(shared_state)

# Kernels
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Agents
coder_agent = ChatCompletionAgent(
    service_id="coder_kernel",
    kernel=coder_kernel,
    name="CoderAgent",
    instructions="Vous êtes le Coder. Mettez à jour le notebook selon les instructions fournies dans les cellules de markdown, de code, et leurs sorties."
)
reviewer_agent = ChatCompletionAgent(
    service_id="reviewer_kernel",
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions="Vous êtes le Reviewer. Validez le notebook en l'exécutant et en analysant les sorties."
)
admin_agent = ChatCompletionAgent(
    service_id="admin_kernel",
    kernel=admin_kernel,
    name="AdminAgent",
    instructions="Vous êtes l'Admin. Approuvez le notebook UNIQUEMENT si le Reviewer a validé les sorties et que le code est fonctionnel. Sinon, demandez des corrections."
)

# GroupChat avec stratégies
termination_strategy = ApprovedBasedTerminationStrategy(shared_state)
selection_strategy = NotebookAwareSelectionStrategy(shared_state)

group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés, group_chat initialisé.")


2025-02-10 07:58:07,822 [DEBUG] Orchestration - Notebook 'Notebook-Template.ipynb' chargé, nb de cellules=11.
2025-02-10 07:58:08,685 [INFO] Orchestration - Agents créés, group_chat initialisé.


## Boucle finale


In [66]:
# Cellule Code : Boucle principale

import asyncio

async def run_conversation():
    try:
        group_chat.history.add_user_message(
            "Bonjour, je veux ajouter du code SPARQL pour DBpedia et afficher un plot Plotly.\n"
            "Mettez à jour la cellule 2 pour inclure le code python d'exemple. Puis validez."
        )

        async for message in group_chat.invoke():
            role = message.role.name
            content = message.content
            logger.info(f"{role.upper()} >>> {content}")
            
            if shared_state.is_approved():
                logger.info("Notebook approuvé - arrêt de la conversation")
                break

    except asyncio.CancelledError:
        logger.warning("Conversation annulée prématurément!")
    finally:
        logger.info(f"Statut final - Approuvé: {shared_state.is_approved()}")
        logger.info("Conversation terminée.")


# Exécution
await run_conversation()


2025-02-10 07:58:16,253 [INFO] Orchestration - ASSISTANT >>> Pour ajouter le code SPARQL pour interroger DBpedia et afficher les résultats avec Plotly, vous pouvez suivre cet exemple. Je vais mettre à jour la cellule 2 en conséquence.

```python
# Cellule 2: Code Python pour interroger DBpedia avec SPARQL et afficher un plot avec Plotly

import requests
import pandas as pd
import plotly.express as px

# SPARQL query to get information about some famous cities
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?city ?population WHERE {
  ?city a dbo:City ;
        dbo:populationTotal ?population .
}
ORDER BY DESC(?population)
LIMIT 10
"""

# Endpoint for DBpedia SPARQL
url = 'http://dbpedia.org/sparql'

# Sending the request
response = requests.get(url, params={'query': query, 'format': 'application/json'})
data = response.json()

# Ext

CancelledError: 